In [1]:
import pandas as pd
import os

In [2]:
# https://www.elections.ca/content.aspx?section=res&dir=rep/off&document=index&lang=e#officialvoting
ids = ['10001', '10002', '10003', '10004', '10005', '10006', '10007', '11001', '11002', '11003', '11004', '12001', '12002', '12003', '12004', '12005', '12006', '12007', '12008', '12009', '12010', '12011', '13001', '13002', '13003', '13004', '13005', '13006', '13007', '13008', '13009', '13010', '24001', '24002', '24003', '24004', '24005', '24006', '24007', '24008', '24009', '24010', '24011', '24012', '24013', '24014', '24015', '24016', '24017', '24018', '24019', '24020', '24021', '24022', '24023', '24024', '24025', '24026', '24027', '24028', '24029', '24030', '24031', '24032', '24033', '24034', '24035', '24036', '24037', '24038', '24039', '24040', '24041', '24042', '24043', '24044', '24045', '24046', '24047', '24048', '24049', '24050', '24051', '24052', '24053', '24054', '24055', '24056', '24057', '24058', '24059', '24060', '24061', '24062', '24063', '24064', '24065', '24066', '24067', '24068', '24069', '24070', '24071', '24072', '24073', '24074', '24075', '24076', '24077', '24078', '35001', '35002', '35003', '35004', '35005', '35006', '35007', '35008', '35009', '35010', '35011', '35012', '35013', '35014', '35015', '35016', '35017', '35018', '35019', '35020', '35021', '35022', '35023', '35024', '35025', '35026', '35027', '35028', '35029', '35030', '35031', '35032', '35033', '35034', '35035', '35036', '35037', '35038', '35039', '35040', '35041', '35042', '35043', '35044', '35045', '35046', '35047', '35048', '35049', '35050', '35051', '35052', '35053', '35054', '35055', '35056', '35057', '35058', '35059', '35060', '35061', '35062', '35063', '35064', '35065', '35066', '35067', '35068', '35069', '35070', '35071', '35072', '35073', '35074', '35075', '35076', '35077', '35078', '35079', '35080', '35081', '35082', '35083', '35084', '35085', '35086', '35087', '35088', '35089', '35090', '35091', '35092', '35093', '35094', '35095', '35096', '35097', '35098', '35099', '35100', '35101', '35102', '35103', '35104', '35105', '35106', '35107', '35108', '35109', '35110', '35111', '35112', '35113', '35114', '35115', '35116', '35117', '35118', '35119', '35120', '35121', '46001', '46002', '46003', '46004', '46005', '46006', '46007', '46008', '46009', '46010', '46011', '46012', '46013', '46014', '47001', '47002', '47003', '47004', '47005', '47006', '47007', '47008', '47009', '47010', '47011', '47012', '47013', '47014', '48001', '48002', '48003', '48004', '48005', '48006', '48007', '48008', '48009', '48010', '48011', '48012', '48013', '48014', '48015', '48016', '48017', '48018', '48019', '48020', '48021', '48022', '48023', '48024', '48025', '48026', '48027', '48028', '48029', '48030', '48031', '48032', '48033', '48034', '59001', '59002', '59003', '59004', '59005', '59006', '59007', '59008', '59009', '59010', '59011', '59012', '59013', '59014', '59015', '59016', '59017', '59018', '59019', '59020', '59021', '59022', '59023', '59024', '59025', '59026', '59027', '59028', '59029', '59030', '59031', '59032', '59033', '59034', '59035', '59036', '59037', '59038', '59039', '59040', '59041', '59042', '60001', '61001', '62001']

year_urls = {
    '2015': 'https://www.elections.ca/res/rep/off/ovr2015app/41/data_donnees/pollresults_resultatsbureau[district].csv',
    '2011': 'https://www.elections.ca/scripts/OVR2011/34/data_donnees/pollresults_resultatsbureau[district].csv',
    '2008': 'https://www.elections.ca/scripts/OVR2008/31/data/pollresults_resultatsbureau[district].csv',
    '2006': 'https://www.elections.ca/Scripts/OVR2006/25/data_donnees/pollresults_resultatsbureau[district].csv',
#     '2004': 'https://www.elections.ca/scripts/OVR2004/23/data/pollbypoll[district].csv'
}

for year, url in year_urls.items():
    print(year)
    dat = pd.DataFrame()
    combined_file_name = f'{year}_election_combined.csv'
    if combined_file_name not in os.listdir():
        for i in ids:
            try:
                temp_file = url.replace('//', '/').replace('/', '-').replace(':', '-').replace('[district]', i)
                if temp_file not in os.listdir('raw_data/'):
                    temp_url = url.replace('[district]', i)
                    temp = pd.read_csv(temp_url, encoding='ISO 8859-1')
                    temp.to_csv(f"raw_data/{temp_file}")
                else:
                    temp = pd.read_csv(f'raw_data/{temp_file}')
                temp.columns = [str(x).split('/')[0] for x in temp.columns]
                dat = dat.append(temp)
            except:
                print(f'Failed at: {year}, {i}')
        dat.to_csv(combined_file_name)

2015
2011
2008
2006


In [8]:
# fix differences in accent use and multiple spellings for NDP
ndp_variants = ['N D P ', 'NDP New Democratic Party', 'NDP-New Democratic Party', 'NDP', 'N.D.P.']
party_column = 'Political Affiliation Name_English'
riding_column = 'Electoral District Name_English'

# clean and combine
combined_dat = pd.DataFrame()
for year in year_urls:
    combined_file = f'./data/{year}_election_combined.csv'
    temp = pd.read_csv(combined_file)
    new_cols = []
    for col in temp.columns:
        if "Unnamed:" in col or "_French" in col:
            temp.drop(col, axis=1, inplace=True)
        else:
            try:
                new_cols += [col.encode('ISO 8859-1').decode('utf-8')]
            except:
                new_cols += [col]
    temp.columns = new_cols
    temp[party_column] = temp[party_column].apply(lambda x: 'NDP' if x in ndp_variants else x)
    temp[riding_column] = temp[riding_column].apply(lambda x: x.replace("\"", ""))
    
    for col in temp.columns:
        if len(temp[col].dropna()) > 0:
            if type(temp[col].dropna().values[0])==str:
                try:
                    temp[col] = temp[col].dropna().apply(lambda x: x.encode('ISO 8859-1').decode('utf-8'))
                except:
                    print(year, col)

    temp['Year'] = year
    combined_dat = combined_dat.append(temp)
    temp.to_csv(f'./data/{combined_file}')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2015 Electoral District Name_English
2015 Polling Station Name
2015 Candidate’s Family Name
2015 Candidate’s First Name
2015 Political Affiliation Name_English


FileNotFoundError: [Errno 2] No such file or directory: './data/./data/2015_election_combined.csv'

In [7]:
combined_dat.to_csv('./combined_data/2006-2015_election_combined.csv')